In [19]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from pathlib import Path
import shutil
import numpy as np
import thor.data as data
import thor.data.dispatch as dispatch
import thor.grid as grid
import thor.track as track
import thor.option as option
import thor.visualize as visualize
import thor.analyze as analyze

notebook_name = "cpol_demo.ipynb"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
# Parent directory for saving outputs
base_local = Path.home() / "THOR_output"
start = "2005-11-13T16:00:00"
end = "2005-11-13T18:00:00"

output_directory = base_local / "runs/cpol_demo_geographic"
options_directory = output_directory / "options"

if output_directory.exists():
    shutil.rmtree(output_directory)

# Create the data_options dictionary
converted_options = {"save": True, "load": False, "parent_converted": None}
cpol_options = data.aura.cpol_data_options(
    start=start, end=end, converted_options=converted_options
)
# Restrict the ERA5 data to a smaller region containing the CPOL radar
lon_range = [129, 133]
lat_range = [-14, -10]
era5_pl_options = data.era5.data_options(
    start=start, end=end, latitude_range=lat_range, longitude_range=lon_range
)
era5_sl_options = data.era5.data_options(
    start=start,
    end=end,
    data_format="single-levels",
    latitude_range=lat_range,
    longitude_range=lon_range,
)
data_options = option.consolidate_options(
    [cpol_options, era5_pl_options, era5_sl_options]
)

dispatch.check_data_options(data_options)
data.option.save_data_options(data_options, options_directory)

altitude = list(np.arange(0, 25e3 + 500, 500))
altitude = [float(alt) for alt in altitude]
grid_options = grid.create_options(name="geographic", altitude=altitude)
grid.check_options(grid_options)
grid.save_grid_options(grid_options, options_directory)

# Create the track_options dictionary
track_options = option.mcs(dataset="cpol")
option.save_track_options(track_options, options_directory)

# Create the display_options dictionary
visualize_options = {
    obj: visualize.option.runtime_options(obj, save=True, style="presentation")
    for obj in ["mcs"]
}
visualize_options["middle_echo"] = visualize.option.runtime_options(
    "middle_echo", save=True, style="presentation", figure_types=["mask"]
)
# visualize.option.save_display_options(visualize_options, options_directory)
visualize_options = None

2024-10-03 13:55:10,592 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_geographic/options/data.yml
2024-10-03 13:55:10,635 - thor.grid - WARNING - Shape not specified. Will attempt to infer from input.
2024-10-03 13:55:10,636 - thor.grid - WARNING - Coordinates not specified. Will attempt to infer from input.
2024-10-03 13:55:10,638 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_geographic/options/grid.yml
2024-10-03 13:55:10,658 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_geographic/options/track.yml


In [21]:
times = data.utils.generate_times(data_options["cpol"])
tracks = track.simultaneous_track(
    times,
    data_options,
    grid_options,
    track_options,
    visualize_options,
    output_directory=output_directory
)

2024-10-03 13:55:11,490 - thor.track - INFO - Beginning thor run. Saving output to /home/ewan/THOR_output/runs/cpol_demo_geographic.
2024-10-03 13:55:11,491 - thor.track - INFO - Beginning simultaneous tracking.
2024-10-03 13:55:11,560 - thor.track - INFO - Processing 2005-11-13T16:00:08.
2024-10-03 13:55:11,561 - thor.data.aura - INFO - Updating cpol dataset for 2005-11-13T16:00:08.
2024-10-03 13:55:11,562 - thor.data.aura - INFO - Converting cpol data from twp10cpolgrid150.b2.20051113.160000.nc
2024-10-03 13:55:11,611 - thor.data.aura - INFO - Creating new geographic grid with spacing 0.025 m, 0.025 m.
2024-10-03 13:55:12,133 - thor.track - INFO - Processing hierarchy level 0.
2024-10-03 13:55:12,134 - thor.track - INFO - Tracking cell.
2024-10-03 13:55:12,184 - thor.match.match - INFO - Matching cell objects.
2024-10-03 13:55:12,186 - thor.match.match - INFO - No previous mask, or no objects in previous mask.
2024-10-03 13:55:12,190 - thor.visualize.runtime - INFO - Generating runti

# Cartesian Coordinates

In [22]:
output_directory = base_local / "runs/cpol_demo_cartesian"
options_directory = output_directory / "options"

if output_directory.exists():
    shutil.rmtree(output_directory)

grid_options = grid.create_options(name="cartesian", regrid=False, altitude=altitude)
grid.check_options(grid_options)
grid.save_grid_options(grid_options, options_directory)
data.option.save_data_options(data_options, options_directory)
option.save_track_options(track_options, options_directory)
# visualize.option.save_display_options(visualize_options, options_directory)

2024-10-03 13:55:34,330 - thor.grid - WARNING - Shape not specified. Will attempt to infer from input.
2024-10-03 13:55:34,333 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_cartesian/options/grid.yml
2024-10-03 13:55:34,349 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_cartesian/options/data.yml
2024-10-03 13:55:34,406 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_cartesian/options/track.yml


In [23]:
times = data.utils.generate_times(data_options["cpol"])
tracks = track.simultaneous_track(
    times,
    data_options,
    grid_options,
    track_options,
    visualize_options,
    output_directory=output_directory,
)

2024-10-03 13:55:41,282 - thor.track - INFO - Beginning thor run. Saving output to /home/ewan/THOR_output/runs/cpol_demo_cartesian.
2024-10-03 13:55:41,283 - thor.track - INFO - Beginning simultaneous tracking.
2024-10-03 13:55:41,354 - thor.track - INFO - Processing 2005-11-13T16:00:08.
2024-10-03 13:55:41,355 - thor.data.aura - INFO - Updating cpol dataset for 2005-11-13T16:00:08.
2024-10-03 13:55:41,356 - thor.data.aura - INFO - Converting cpol data from twp10cpolgrid150.b2.20051113.160000.nc
2024-10-03 13:55:41,547 - thor.track - INFO - Processing hierarchy level 0.
2024-10-03 13:55:41,548 - thor.track - INFO - Tracking cell.
2024-10-03 13:55:41,572 - thor.match.match - INFO - Matching cell objects.
2024-10-03 13:55:41,573 - thor.match.match - INFO - No previous mask, or no objects in previous mask.
2024-10-03 13:55:41,576 - thor.visualize.runtime - INFO - Generating runtime visualizations.
2024-10-03 13:55:41,577 - thor.track - INFO - Tracking middle_echo.
2024-10-03 13:55:41,588 

# Analysis

In [25]:
analysis_options = analyze.mcs.analysis_options()
analyze.mcs.process_velocities(output_directory)
analyze.mcs.quality_control(output_directory, analysis_options)
analyze.mcs.classify_all(output_directory)

2024-10-03 13:56:01,820 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/cpol_demo_cartesian/analysis/velocities.csv
2024-10-03 13:56:01,824 - thor.write.attribute - DEBUG - Saving attribute metadata to /home/ewan/THOR_output/runs/cpol_demo_cartesian/analysis/velocities.yml
2024-10-03 13:56:02,184 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/cpol_demo_cartesian/analysis/quality.csv
2024-10-03 13:56:02,188 - thor.write.attribute - DEBUG - Saving attribute metadata to /home/ewan/THOR_output/runs/cpol_demo_cartesian/analysis/quality.yml
2024-10-03 13:56:02,308 - thor.write.attribute - DEBUG - Writing attribute dataframe to /home/ewan/THOR_output/runs/cpol_demo_cartesian/analysis/classification.csv
2024-10-03 13:56:02,310 - thor.write.attribute - DEBUG - Saving attribute metadata to /home/ewan/THOR_output/runs/cpol_demo_cartesian/analysis/classification.yml


,,stratiform_offset,inflow,relative_stratiform_offset,tilt,propagation
time,universal_id,,,,,
2005-11-13 16:00:08,1,trailing,front,trailing,up-shear,down-shear
2005-11-13 16:10:23,1,trailing,front,trailing,up-shear,down-shear
2005-11-13 16:20:09,1,trailing,front,trailing,up-shear,down-shear
2005-11-13 16:30:09,1,trailing,front,trailing,up-shear,down-shear
2005-11-13 16:40:09,1,trailing,front,trailing,up-shear,down-shear
2005-11-13 16:50:08,1,trailing,front,left,up-shear,shear-perpendicular
2005-11-13 17:00:08,1,trailing,front,left,up-shear,shear-perpendicular
2005-11-13 17:10:23,1,trailing,left,left,up-shear,shear-perpendicular
2005-11-13 17:20:09,1,trailing,left,left,up-shear,shear-perpendicular


In [26]:
figure_options = visualize.option.horizontal_attribute_options(
    "mcs_velocity_analysis", style="presentation"
)
start_time = np.datetime64("2005-11-13T16:00")
end_time = np.datetime64("2005-11-13T19:00")
visualize.attribute.mcs_series(output_directory, start_time, end_time, figure_options)

2024-10-03 13:56:02,536 - thor.data.aura - INFO - Converting cpol data from twp10cpolgrid150.b2.20051113.160000.nc
2024-10-03 13:56:02,701 - thor.visualize.horizontal - INFO - No figsize provided. Using default.
2024-10-03 13:56:02,917 - thor.visualize.horizontal - DEBUG - Plotting boundary.
2024-10-03 13:56:02,943 - thor.visualize.horizontal - DEBUG - Plotting boundary.
2024-10-03 13:56:03,106 - thor.visualize.attribute - DEBUG - Saving mcs_velocity_analysis figure for 2005-11-13T16:00:08.000000000.
2024-10-03 13:56:05,136 - thor.data.aura - INFO - Converting cpol data from twp10cpolgrid150.b2.20051113.161000.nc
2024-10-03 13:56:05,311 - thor.visualize.horizontal - INFO - No figsize provided. Using default.
2024-10-03 13:56:05,459 - thor.visualize.horizontal - DEBUG - Plotting boundary.
2024-10-03 13:56:05,497 - thor.visualize.horizontal - DEBUG - Plotting boundary.
2024-10-03 13:56:05,662 - thor.visualize.attribute - DEBUG - Saving mcs_velocity_analysis figure for 2005-11-13T16:10:23

In [27]:
import thor.test.test_aura as test_aura

In [12]:
test_aura.test_cpol()

2024-10-03 13:39:03,058 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_geographic/options/data.yml
2024-10-03 13:39:03,114 - thor.grid - WARNING - Shape not specified. Will attempt to infer from input.
2024-10-03 13:39:03,115 - thor.grid - WARNING - Coordinates not specified. Will attempt to infer from input.
2024-10-03 13:39:03,117 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_geographic/options/grid.yml
2024-10-03 13:39:03,141 - thor.option - DEBUG - Saving options to /home/ewan/THOR_output/runs/cpol_demo_geographic/options/track.yml
2024-10-03 13:39:03,177 - thor.track - INFO - Beginning thor run. Saving output to /home/ewan/THOR_output/runs/cpol_demo_geographic.
2024-10-03 13:39:03,178 - thor.track - INFO - Beginning simultaneous tracking.
2024-10-03 13:39:03,282 - thor.track - INFO - Processing 2005-11-13T14:00:09.
2024-10-03 13:39:03,288 - thor.data.aura - INFO - Updating cpol dataset for 2005-11-13T14:00:09.
2024

KeyError: 3000